In [6]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bson.json_util import dumps

# Import base64 for encoding of image
import base64

# Import CRUD object 
from myModule import AnimalShelter

# Define CSS style variables   
HEADER_IMAGE = {'width':'196px'}
TEXT_CENTER = {'text-align':'center'}
TABLE_CELL = {'text-align': 'left', 'min-width': '150px'}
MAP_CONTAINER = {'float':'left',
                 'width': '500px', 
                 'height': '470px', 
                 'margin-top':'10px', 
                 'margin-right':'10px', 
                 'padding':'5px', 
                 'border':'1px solid #e3e3e3',
                 'border-radius': '5px',
                 'box-shadow': '5px 5px 5px #ccc'}
GRAPH_CONTAINER = {'float':'left',
                   'width':'500px', 
                   'margin-top':'10px', 
                   'margin-right':'10px',
                   'padding':'5px',
                   'border':'1px solid #e3e3e3',
                   'border-radius': '5px',
                   'box-shadow': '5px 5px 5px #ccc'}
CLEAR_FIX = {'clear':'both'}
BOLD_TEXT = {'font-weight':'700'}

# Define pagination variables
PAGE_SIZE = 5

# Define filter variables
FILTER_NONE = {}

FILTER_WATER = {'$or':
                [{"breed":"Labrador Retriever Mix"},
                 {"breed":"Chesapeake Bay Retriever"},
                 {"breed":"Newfoundland"}
                ],
                "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156},
                "sex_upon_outcome":"Intact Female"}

FILTER_MOUNTAIN = {'$or':                 
                   [{"breed":"German Shepherd"},
                    {"breed":"Alaskan Malamute"},
                    {"breed":"Old English Sheepdog"},
                    {"breed":"Siberian Husky"},
                    {"breed":"Rottweiler"}
                   ],
                   "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156},
                   "sex_upon_outcome":"Intact Male"}

FILTER_DISASTER = {'$or':
                   [{"breed":"Doberman Pinscher"},
                    {"breed":"German Shepherd"},
                    {"breed":"Golden Retriever"},
                    {"breed":"Bloodhound"},
                    {"breed":"Rottweiler"}
                   ],
                   "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156},
                   "sex_upon_outcome":"Intact Male"}

FILTER_ELDERLY = {"age_upon_outcome_in_weeks":{'$gte':782}} 

# Instantiate CRUD object
animalShelter = AnimalShelter()

# Populate data frame
cursor = animalShelter.read(FILTER_NONE)
df = pd.DataFrame.from_records(list(cursor))
df[' index'] = range(1, len(df) + 1)

#Generate base64 encoded image in memory
encoded_image = base64.b64encode(open('GraziosoSalvareLogo.png', 'rb').read())

app = JupyterDash('SimpleExample')
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
# Header
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style = HEADER_IMAGE),
        html.H3('LaNise Essick - CS-340 Client/Server Development')       
    ], style=TEXT_CENTER), 
    
# Radio Button List
    html.Div([
        dcc.RadioItems(
        id = 'rdoList',
        options = [           
            {'label': 'Water Rescue', 'value': 'WATER'},
            {'label': 'Mountain Rescue', 'value': 'MOUNTAIN'},
            {'label': 'Disaster Rescue ', 'value': 'DISASTER'},
            {'label': 'Elderly (15+) ', 'value': 'ELDERLY'},  
            {'label': 'Reset', 'value': 'ALL'},
        ],
        value = 'WATER',
        labelStyle = {'display': 'inline-block'}
    ),
    ]),
    html.Br(),
      
# Table & Pagination
    html.Div(id='table-output', children = [
            dash_table.DataTable(
                id='datatable-id',
                columns=[
                    {"name": i, "id": i} for i in df.columns
                ])]),
    html.Br(),
    html.Br(),
    html.Br(),
    html.Div(children = [
        html.Div(['Map:'], style=BOLD_TEXT),
        html.Div(id='map-id')
    ], style=MAP_CONTAINER),
    html.Div(children = [
        html.Div(['Outcomes:'], style=BOLD_TEXT),
        dcc.Graph(id="pie-chart")
    ], style=GRAPH_CONTAINER),
    html.Div(style=CLEAR_FIX),
    html.Br(),
    html.Br(),
    html.Br(),
    
]) # End of app
    
# Radio Button List Callbacks
@app.callback(
    dash.dependencies.Output('table-output', 'children'),
    [dash.dependencies.Input('rdoList', 'value')]
    )
def update_output(value):

    # Generate data frame based on radio button list selection
    if value == 'ALL':        
        df = pd.DataFrame.from_records(animalShelter.read(FILTER_NONE))   
    elif value == 'WATER':        
        df = pd.DataFrame.from_records(animalShelter.read(FILTER_WATER))      
    elif value == 'MOUNTAIN':      
        df = pd.DataFrame.from_records(animalShelter.read(FILTER_MOUNTAIN))      
    elif value == 'DISASTER':
        df = pd.DataFrame.from_records(animalShelter.read(FILTER_DISASTER))
    elif value == 'ELDERLY':
        df = pd.DataFrame.from_records(animalShelter.read(FILTER_ELDERLY))
        
    return [
        dash_table.DataTable(
            id = 'datatable-id',
            columns = [{"name": i, "id": i} for i in df.columns],
            data = df.to_dict('records'),
            style_cell = TABLE_CELL,
            style_data_conditional=[{'if': {'row_index': 'odd'},'background-color': '#f3f3f3'}],
            style_header={'background-color': '#e3e3e3','font-weight': 'bold'},
            page_action = 'native',
            page_current = 0,
            page_size = PAGE_SIZE)        
       ]

# Graph Callbacks - - - - - - - - - - - - - - - - - - - - -   
@app.callback(
    Output("pie-chart", "figure"),
    [Input("datatable-id", "data")]
)
def generate_chart(data):
    dff = pd.DataFrame.from_dict(data)
    fig = px.pie(dff,names='outcome_type')
    return fig

# Map Callbacks - - - - - - - - - - - - - - - - - - - - -   
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    # Parse values out of data frame
    posX = [str(i) for i in dff.iloc[0:5,9].values]
    posY = [str(i) for i in dff.iloc[0:5,10].values]   
    animal_ids = dff.iloc[0:5,0].values
    animal_names = dff.iloc[0:5,5].values
    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '500px', 'height': '450px'}, center=[posX[0],posY[0]], zoom=8, children=[
            dl.TileLayer(id="base-layer-id"),

            # Animal 0
            dl.Marker(position=[posX[0],posY[0]], children=[
                dl.Tooltip([
                    html.Span(['ID: '], style=BOLD_TEXT),
                    html.Span(animal_ids[0]),
                    html.Br(),
                    html.Span(['Name: '], style=BOLD_TEXT),
                    html.Span(animal_names[0])
                ])
            ]),
            
            # Animal 1
            dl.Marker(position=[posX[1],posY[1]], children=[
                dl.Tooltip([
                    html.Span(['ID: '], style=BOLD_TEXT),
                    html.Span(animal_ids[1]),
                    html.Br(),
                    html.Span(['Name: '], style=BOLD_TEXT),
                    html.Span(animal_names[1])
                ])
            ]),
            
            # Animal 2
            dl.Marker(position=[posX[2],posY[2]], children=[
                dl.Tooltip([
                    html.Span(['ID: '], style=BOLD_TEXT),
                    html.Span(animal_ids[2]),
                    html.Br(),
                    html.Span(['Name: '], style=BOLD_TEXT),
                    html.Span(animal_names[2])
                ])
            ]),
            
            # Animal 3
            dl.Marker(position=[posX[3],posY[3]], children=[
                dl.Tooltip([
                    html.Span(['ID: '], style=BOLD_TEXT),
                    html.Span(animal_ids[3]),
                    html.Br(),
                    html.Span(['Name: '], style=BOLD_TEXT),
                    html.Span(animal_names[3])
                ])
            ]),
            
            # Animal 4
            dl.Marker(position=[posX[4],posY[4]], children=[
                dl.Tooltip([
                    html.Span(['ID: '], style=BOLD_TEXT),
                    html.Span(animal_ids[4]),
                    html.Br(),
                    html.Span(['Name: '], style=BOLD_TEXT),
                    html.Span(animal_names[4])
                ])
            ]),
            
        ])
    ]


# Execution starts here
app.run_server(debug=True)


{'rec_num': 2, 'age_upon_outcome': '1 year', 'animal_id': 'A725717', 'animal_type': 'Cat', 'breed': 'Domestic Shorthair Mix', 'color': 'Silver Tabby', 'date_of_birth': '2015-05-02', 'datetime': '2016-05-06 10:49:00', 'monthyear': '2016-05-06T10:49:00', 'name': '', 'outcome_subtype': 'SCRP', 'outcome_type': 'Transfer', 'sex_upon_outcome': 'Spayed Female', 'location_lat': 30.6525984560228, 'location_long': -97.7419963476444, 'age_upon_outcome_in_weeks': 52.9215277777778}
{'rec_num': 3, 'age_upon_outcome': '2 years', 'animal_id': 'A716330', 'animal_type': 'Dog', 'breed': 'Chihuahua Shorthair Mix', 'color': 'Brown/White', 'date_of_birth': '2013-11-18', 'datetime': '2015-12-28 18:43:00', 'monthyear': '2015-12-28T18:43:00', 'name': 'Frank', 'outcome_subtype': '', 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Neutered Male', 'location_lat': 30.7595748121648, 'location_long': -97.5523753807133, 'age_upon_outcome_in_weeks': 110.111408730159}
{'rec_num': 5, 'age_upon_outcome': '2 years', 'ani

{'rec_num': 2456, 'age_upon_outcome': '7 years', 'animal_id': 'A661868', 'animal_type': 'Cat', 'breed': 'Russian Blue Mix', 'color': 'Blue', 'date_of_birth': '2006-08-26', 'datetime': '2013-11-15 11:18:00', 'monthyear': '2013-11-15T11:18:00', 'name': 'Purrsimmon', 'outcome_subtype': 'Foster', 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Spayed Female', 'location_lat': 30.3736011428879, 'location_long': -97.4459324978572, 'age_upon_outcome_in_weeks': 376.924404761905}
{'rec_num': 2457, 'age_upon_outcome': '2 days', 'animal_id': 'A706755', 'animal_type': 'Cat', 'breed': 'Domestic Shorthair', 'color': 'Gray Tabby', 'date_of_birth': '2015-07-01', 'datetime': '2015-07-03 11:04:00', 'monthyear': '2015-07-03T11:04:00', 'name': '', 'outcome_subtype': 'Partner', 'outcome_type': 'Transfer', 'sex_upon_outcome': 'Intact Male', 'location_lat': 30.4571001713355, 'location_long': -97.4655383100908, 'age_upon_outcome_in_weeks': 0.351587301587302}
{'rec_num': 2458, 'age_upon_outcome': '1 month', 'a

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'rec_num': 36, 'age_upon_outcome': '6 months', 'animal_id': 'A706953', 'animal_type': 'Dog', 'breed': 'Labrador Retriever Mix', 'color': 'Yellow', 'date_of_birth': '2014-12-06', 'datetime': '2015-07-06 11:33:00', 'monthyear': '2015-07-06T11:33:00', 'name': '', 'outcome_subtype': 'Medical', 'outcome_type': 'Euthanasia', 'sex_upon_outcome': 'Intact Female', 'location_lat': 30.5480802368633, 'location_long': -97.2969969058957, 'age_upon_outcome_in_weeks': 30.3544642857143}
{'rec_num': 732, 'age_upon_outcome': '2 years', 'animal_id': 'A749782', 'animal_type': 'Dog', 'breed': 'Labrador Retriever Mix', 'color': 'Tan/White', 'date_of_birth': '2015-05-19', 'datetime': '2017-07-25 14:59:00', 'monthyear': '2017-07-25T14:59:00', 'name': '*Catalina', 'outcome_subtype': '', 'outcome_type': 'Return to Owner', 'sex_upon_outcome': 'Intact Female', 'location_lat': 30.6138310636757, 'location_long': -97.5752164857665, 'age_upon_outcome_in_weeks': 114.089186507937}
{'rec_num': 1121, 'age_upon_outcome': 